In [1]:
import numpy as np

In [4]:
splits=np.load("splits.npz")

In [9]:
dict(splits)["idx_val"]

array([ 430,  512,   70,  395,  737,  630,   11,  709,  478,  894,  719,
        842,  392,  680,  131,  322,  626,  292,  916,  197,  560,  847,
        966, 1002,  216,  531,  426,  312,  812,   26,  319, 1028,  290,
       1042, 1012,  451,  155,  532,  948,  780, 1071,  820,   56,  399,
        708,  207,  841,  957,  553,  326,  794,  752,  444, 1052,  435,
        173,  431,  503, 1121, 1059,   47,  689,  393,   86, 1084,  296,
        439,  667,  106,  582,  584,  814,  117,   57,  936,  668,  730,
        423, 1111,  251,  448,  609,  684, 1017,  900,  829, 1081,  282,
         64, 1001,  146,  979,  618,  872,  977,  886,   42,   79,  883,
        845], dtype=int64)